<a href="https://www.kaggle.com/code/anshulyadav05/titanic-logisticregression?scriptVersionId=103800298" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.preprocessing import OrdinalEncoder
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression as lr
from sklearn.model_selection import train_test_split
import seaborn as sns


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


# **Reading the TITANIC dataset from input**

In [2]:
train_data=pd.read_csv('../input/titanic/train.csv')
test_data=pd.read_csv('../input/titanic/test.csv')


train_data.info()
print('\n\n\n')
test_data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB




<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pcla

# Dropping the features for model evaluation

In [3]:
drop_features=['Name','Ticket','Cabin','Embarked']
train_data=train_data.drop(drop_features,axis=1)
test_data=test_data.drop(drop_features,axis=1)

In [4]:
train_data.info()
print('\n\n\n')
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Sex          891 non-null    object 
 4   Age          714 non-null    float64
 5   SibSp        891 non-null    int64  
 6   Parch        891 non-null    int64  
 7   Fare         891 non-null    float64
dtypes: float64(2), int64(5), object(1)
memory usage: 55.8+ KB




<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Sex          418 non-null    object 
 3   Age          332 non-null    float64
 4   SibSp        418 non-null    int64  
 5   Parch 

# Looking for dublicated,null Values

In [5]:
print('\n Passenger Classes for train data: ',train_data.Pclass.unique())

print('\n duplicate data :',train_data.duplicated().sum())

print('\n null data: ',train_data.isnull().sum())


 Passenger Classes for train data:  [3 1 2]

 duplicate data : 0

 null data:  PassengerId      0
Survived         0
Pclass           0
Sex              0
Age            177
SibSp            0
Parch            0
Fare             0
dtype: int64


In [6]:
train_data['Age']

0      22.0
1      38.0
2      26.0
3      35.0
4      35.0
       ... 
886    27.0
887    19.0
888     NaN
889    26.0
890    32.0
Name: Age, Length: 891, dtype: float64

# Firstly Ordinal Encoding object value column i.e, 'Sex' column

In [7]:
s=(train_data.dtypes=='object')
object_cols=list(s[s].index)
oe=OrdinalEncoder()
label_train_data=train_data.copy()
label_test_data=test_data.copy()
label_train_data[object_cols]=oe.fit_transform(train_data[object_cols])
label_test_data[object_cols]=oe.fit_transform(test_data[object_cols])
label_train_data.head()
label_train_data.isnull().sum()

print(label_train_data[-4:])
      
print(label_test_data[-4:])

     PassengerId  Survived  Pclass  Sex   Age  SibSp  Parch   Fare
887          888         1       1  0.0  19.0      0      0  30.00
888          889         0       3  0.0   NaN      1      2  23.45
889          890         1       1  1.0  26.0      0      0  30.00
890          891         0       3  1.0  32.0      0      0   7.75
     PassengerId  Pclass  Sex   Age  SibSp  Parch      Fare
414         1306       1  0.0  39.0      0      0  108.9000
415         1307       3  1.0  38.5      0      0    7.2500
416         1308       3  1.0   NaN      0      0    8.0500
417         1309       3  1.0   NaN      1      1   22.3583


# Imputing the null values in 'Age' column

In [8]:
cols_for_impute=list(train_data['Age'].index)
print(cols_for_impute)
#from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer
imputer=SimpleImputer()
label_train_data=pd.DataFrame(imputer.fit_transform(label_train_data))
label_train_data.columns=train_data.columns
label_test_data=pd.DataFrame(imputer.fit_transform(label_test_data))
label_test_data.columns=test_data.columns
label_train_data.info()

print('\n\n\n')
label_test_data.info()
print('\n\n\n')
label_train_data.info()


[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221,

In [9]:
label_test_data.head

<bound method NDFrame.head of      PassengerId  Pclass  Sex       Age  SibSp  Parch      Fare
0          892.0     3.0  1.0  34.50000    0.0    0.0    7.8292
1          893.0     3.0  0.0  47.00000    1.0    0.0    7.0000
2          894.0     2.0  1.0  62.00000    0.0    0.0    9.6875
3          895.0     3.0  1.0  27.00000    0.0    0.0    8.6625
4          896.0     3.0  0.0  22.00000    1.0    1.0   12.2875
..           ...     ...  ...       ...    ...    ...       ...
413       1305.0     3.0  1.0  30.27259    0.0    0.0    8.0500
414       1306.0     1.0  0.0  39.00000    0.0    0.0  108.9000
415       1307.0     3.0  1.0  38.50000    0.0    0.0    7.2500
416       1308.0     3.0  1.0  30.27259    0.0    0.0    8.0500
417       1309.0     3.0  1.0  30.27259    1.0    1.0   22.3583

[418 rows x 7 columns]>

# Logistic Regression Model

In [10]:
x=label_train_data.drop(['Survived'],axis=1)
y=label_train_data.Survived

In [11]:
model=lr(random_state=0,max_iter=250).fit(x,y)
model.score(x,y)

0.797979797979798

In [12]:
result=model.predict(label_test_data)

print(len(result))
result

418


array([0., 0., 0., 0., 1., 0., 1., 0., 1., 0., 0., 0., 1., 0., 1., 1., 0.,
       0., 1., 0., 0., 0., 1., 1., 1., 0., 1., 0., 0., 0., 0., 0., 0., 1.,
       0., 0., 1., 1., 0., 0., 0., 1., 0., 1., 1., 0., 0., 0., 1., 1., 1.,
       0., 1., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 1., 1., 1., 0.,
       0., 1., 1., 0., 1., 1., 1., 1., 0., 1., 0., 1., 0., 0., 0., 0., 0.,
       0., 1., 1., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0.,
       0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 0., 0., 1., 0.,
       1., 1., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
       0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 1., 0., 1., 0., 1., 0., 0.,
       1., 0., 0., 1., 1., 0., 1., 1., 0., 1., 0., 0., 1., 0., 0., 1., 1.,
       0., 0., 0., 0., 0., 1., 1., 0., 1., 1., 0., 0., 1., 0., 1., 0., 1.,
       0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 1., 0., 1., 1., 0., 0., 1.,
       0., 1., 1., 0., 1., 0., 0., 0., 0., 1., 0., 0., 1., 0., 1., 0., 1.,
       0., 1., 0., 1., 1.

# Saving the result in'submission.csv' file

In [13]:
column=test_data.PassengerId
df=pd.DataFrame(column,columns=['PassengerId'])
df1=pd.DataFrame(result,columns=['Survived'])

In [14]:
output=pd.concat([df,df1],axis=1,join='inner')
output.to_csv('submission.csv',index=False)

output.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Survived     418 non-null    float64
dtypes: float64(1), int64(1)
memory usage: 6.7 KB
